In [1]:
%%capture
!pip install transformers datasets
!python -m pip install huggingface_hub


In [2]:
MINE = "jimregan/wav2vec2-swedish-riksdag-200h"


In [3]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
from datasets import load_dataset

cv6 = load_dataset("jimregan/sbtal_riksdag_asr", split="test", use_auth_token=True)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/73826 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/478 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/453 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/jimregan___parquet/jimregan--sbtal_riksdag_asr-8175b7a674e1b59a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


In [ ]:
cv6

Dataset({
    features: ['audio', 'text'],
    num_rows: 453
})

In [5]:
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 84.1 MB/s eta 0:00:00


In [7]:
from datasets import load_metric
wer_metric = load_metric("wer")

<ipython-input-7-7b292a9c8d88>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [8]:
from transformers import AutoModelForCTC, Wav2Vec2Processor

mymodel = AutoModelForCTC.from_pretrained(MINE)
myprocessor = Wav2Vec2Processor.from_pretrained(MINE)


In [9]:
!pip install torchaudio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import torch
import torchaudio
import re

def speech_file_to_array_fn(batch):
    chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“]'
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    return batch

In [11]:
cv6 = cv6.map(speech_file_to_array_fn)
#cv11 = cv11.map(speech_file_to_array_fn)

Map:   0%|          | 0/453 [00:00<?, ? examples/s]

In [12]:
cv6[0]

{'audio': {'path': 'sweterror-01-1-a_12075_17898.wav',
  'array': array([ 7.62939453e-04, -6.10351562e-05,  1.64794922e-03, ...,
         -3.05175781e-05,  0.00000000e+00, -3.05175781e-05]),
  'sampling_rate': 16000},
 'text': 'i betänkandet tillstyrker utskottet regeringens förslag om anslag för utgiftsområde tre',
 'sentence': 'i betänkandet tillstyrker utskottet regeringens förslag om anslag för utgiftsområde tre'}

In [13]:
import numpy as np

def evaluate(batch):
    audio_dict = batch["audio"][0]["array"]
    inputs = myprocessor(audio_dict, sampling_rate=16_000, return_tensors="pt", padding=True)
    mymodel.to("cuda")
    with torch.no_grad():
        logits = mymodel(inputs.input_values.to("cuda"), attention_mask=inputs.attention_mask.to("cuda")).logits    
    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_strings"] = myprocessor.batch_decode(pred_ids)
    return batch
result = cv6.map(evaluate, batched=True, batch_size=1)

Map:   0%|          | 0/453 [00:00<?, ? examples/s]

In [14]:
print("WER: {:2f}".format(100 * wer_metric.compute(predictions=result["pred_strings"], references=result["sentence"])))

WER: 16.378316
